In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [ ]:
file_path = 'input_game.csv'
df = pd.read_csv(file_path)

In [ ]:
df.replace({'TRUST': 0, 'CHEAT': 1}, inplace=True)

In [ ]:
df

In [ ]:
number_of_rounds = []

In [ ]:
total_rounds_per_game = df.groupby('game_id')['turn'].count()

In [ ]:
total_rounds_per_game

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(total_rounds_per_game.index, total_rounds_per_game.values, color='skyblue')
plt.xlabel('Game ID')
plt.ylabel('Number of Rounds')
plt.title('Number of Rounds Played for Each Game')
plt.xticks(total_rounds_per_game.index)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Another visualization of the distribution
plt.hist(df['turn'], bins=range(min(df['turn']), max(df['turn']) + 1), edgecolor='black')
plt.xlabel('Number of Rounds')
plt.ylabel('Frequency')
plt.title('Distribution of Rounds Played')
plt.show()

In [ ]:
#Uniform Distribution between [40,80]

In [ ]:
def create_models(p1_moves, p2_moves):
    moves_player1 = np.array(p1_moves[:-1])
    moves_player2 = np.array(p2_moves[:-1])

    # Prepare data for LSTM
    X_moves = np.column_stack((moves_player1, moves_player2))

    # Prepare labels for player 1 and player 2
    y_player1 = np.array(p1_moves[1:])  # Next move for player 1
    y_player2 = np.array(p2_moves[1:])  # Next move for player 2

    model_player1 = Sequential([
        LSTM(64, input_shape=(X_moves.shape[1], 1)),
        Dense(1, activation='sigmoid')
    ])

    model_player2 = Sequential([
        LSTM(64, input_shape=(X_moves.shape[1], 1)),
        Dense(1, activation='sigmoid')
    ])

    model_player1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_player2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train models multiple times
    for _ in range(101):
        model_player1.fit(X_moves, y_player1, epochs=10, batch_size=32, verbose=0)
        model_player2.fit(X_moves, y_player2, epochs=10, batch_size=32, verbose=0)

    return model_player1, model_player2

In [ ]:
grouped_df = df.groupby('game_id').agg({'p1_action': list, 'p2_action': list}).reset_index()
grouped_df.rename(columns={'p1_action': 'input', 'p2_action': 'target'}, inplace=True)

In [ ]:
p1 = grouped_df.iloc[0, 1] +grouped_df.iloc[317, 1]
p2 = grouped_df.iloc[0, 2] +grouped_df.iloc[317, 2]

In [ ]:
m1, m2 = create_models(p1,p2)

In [ ]:
p1_ = grouped_df.iloc[948, 1]
p2_ = grouped_df.iloc[948, 2]

In [ ]:
moves_player1 = np.array(p1_[:-1])
moves_player2 = np.array(p2_[:-1])

# Preprocessing
# Combine moves of both players into one array
X_new_moves = np.column_stack((moves_player1, moves_player2))

# Prepare labels for player 1 and player 2
y_player1 = np.array(p1_[1:])  # Next move for player 1
y_player2 = np.array(p2_[1:])  # Next move for player 2

In [ ]:
predictions_player1 = m1.predict(X_new_moves)

In [ ]:
y_player1

In [ ]:
preds  = (predictions_player1.reshape(1,-1)[0] > 0.5).astype(int)

In [ ]:
# PART II

In [ ]:
def calculate_similarity(actions1, actions2):
    return np.sum(actions1 == actions2)

In [ ]:
# Rename columns to match the code
df.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)

# Sample data generation
grouped_df = df.groupby('game_id').agg({'player_id': 'first', 'p1_action': list, 'p2_action': list}).reset_index()

In [ ]:
"""
# Create models and make predictions
models = {}
for index, row in grouped_df.iterrows():
    p1_actions = row['p1_action']
    p2_actions = row['p2_action']
    model_p1, model_p2 = create_models(p1_actions, p2_actions)
    models[row['game_id']] = (model_p1, model_p2)
"""

In [ ]:
# Calculate similarity and group by player_id
grouped_by_strategy = {}
for game_id, (model_p1, model_p2) in models.items():
    grouped_by_strategy.setdefault(game_id, [])
    p1_actions = grouped_df[grouped_df['game_id'] == game_id]['p1_action'].iloc[0]
    p2_actions = grouped_df[grouped_df['game_id'] == game_id]['p2_action'].iloc[0]
    X_new_moves = np.column_stack((np.array(p1_actions[:-1]), np.array(p2_actions[:-1])))

    predicted_p1 = model_p1.predict(X_new_moves)
    predicted_p2 = model_p2.predict(X_new_moves)

    similarity_p1 = calculate_similarity(np.round(predicted_p1.flatten()), np.array(p1_actions[1:]))
    similarity_p2 = calculate_similarity(np.round(predicted_p2.flatten()), np.array(p2_actions[1:]))

    grouped_by_strategy[game_id].append((similarity_p1, similarity_p2))

# Calculate strategy groups based on similarity
strategy_groups = {}
for game_id, similarities in grouped_by_strategy.items():
    p1_id = grouped_df[grouped_df['game_id'] == game_id]['p1_id'].iloc[0]
    p2_id = grouped_df[grouped_df['game_id'] == game_id]['p2_id'].iloc[0]
    min_id = min(p1_id, p2_id)

    if min_id in strategy_groups:
        strategy_groups[min_id].append(game_id)
    else:
        strategy_groups[min_id] = [game_id]

# Sort the strategies and players within each strategy
sorted_strategy_groups = sorted(strategy_groups.items(), key=lambda x: x[0])
final_groups = [[player_id] + sorted(game_ids) for player_id, game_ids in sorted_strategy_groups]
